In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


In [ ]:
RANDOM_SEED = 2_055

In [ ]:
# Define paths
base_dir = './../data/face_age_transformed'
train_dir = base_dir

# Image Data Generator with rescaling
train_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2, seed=RANDOM_SEED)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),  # Adjust based on your model input size
    batch_size=32,
    class_mode='binary',  # Assuming binary classification: adult vs minor
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)
